<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Hugging_Face_Sara_FineTuning_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy


In [27]:
# Imports
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer

In [28]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"

In [29]:
# PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [30]:
# Text in Chunks teilen
def split_text(text, chunk_size=300, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


 PDF in 2969 Chunks unterteilt!


In [31]:
# Lade das Modell
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_relevant_chunks(query, top_k=3):
    # Berechne Embedding der Frage
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)

    # Berechne Embeddings für alle Chunks
    chunk_embeddings = np.array(embedding_model.encode(chunks, batch_size=64, convert_to_numpy=True))

    # Berechne die Cosinus-Ähnlichkeit zwischen der Frage und den Chunks
    similarities = np.dot(chunk_embeddings, query_embedding) / (np.linalg.norm(chunk_embeddings, axis=1) * np.linalg.norm(query_embedding))

    # Finde die Top-K relevantesten Chunks
    top_k_indices = similarities.argsort()[-top_k:][::-1]

    relevant_chunks = [chunks[i] for i in top_k_indices]

    # Rückgabe der Chunks als Kontext
    return relevant_chunks



In [32]:
# FAISS Vektordatenbank aufsetzen
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)
print(" FAISS Vektordatenbank erstellt!")


 FAISS Vektordatenbank erstellt!


In [33]:
# Ähnlichkeitssuche in FAISS
def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]


In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

!huggingface-cli login

os.environ["HUGGINGFACE_API_KEY"] = "HF2"
model_name = "mistralai/Mistral-7B-v0.1"  # Alternativ: "meta-llama/Llama-2-7b-chat-hf"

# Tokenizer & Modell mit explizitem Token laden
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")


def generate_answer(query):
    relevant_chunks = get_relevant_chunks(query, top_k=3)
    context = "\n".join(relevant_chunks)
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n\n{context}\n\nFrage: {query}\nAntwort:"

    response = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.7)
    return response[0]["generated_text"]



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# Test
frage = "Welche Schutzrechte gibt es für werdende Mütter im Arbeitsrecht?"
antwort = generate_answer(frage)
print("Antwort:", antwort)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Antwort: Beantworte die Frage basierend auf diesem Kontext:

Schutzrechte zugunsten der (rauchenden) Arbeitnehmer eingreifen. Der AN macht Dis­
kriminierung sowie eine Verletzung seines Persönlichkeitsrechts geltend. Ersteres schei­
tert ersichtlich daran, dass kein in § 1 AGG genannter Benachteiligungsgrund vorliegt. 
Letzteres erfordert einen rechtswidrigen
s://doi.org/10.1007/978-3-662-65573-3_1
1
Einleitung
1.1	
Schutzbereich
Das Arbeitsrecht umfasst das Recht der abhängigen Arbeit. Abhängige Arbeit wird 
von Arbeitnehmern ausgeführt. Das heißt, die Ausübung selbstständiger Tätigkeit bewegt 
sich weitgehend außerhalb des arbeitsrechtlichen Schutzber
r ethnischen Herkunft, 
z. B. in Bezug auf bestimmte Weisungen des Arbeitgebers (BAG – 8 AZR 364/11).
1.8  Schutz vor Diskriminierung im Arbeitsrecht

24
Kontrollfragen
	1.	 In welche beiden großen Bereiche wird das Arbeitsrecht eingeteilt und wie werden 
die Bereiche definiert?
	2.	 Welche Bedeutun

Frage: Welche Schutzrechte gibt es 